## Requirements

* one set for eval during training and one set for final test
* roughly class balanced
* shuffled?

In [ ]:
import numpy.random as nr
from random import shuffle
import math
import tensorflow as tf
from object_detection.utils import label_map_util
from CNNRobotLocalisation.Utils.file_utils import *

In [ ]:
STAGE = "Second"
if STAGE == "First":
    PATH = '/media/data/LocalizationDataNew/Output/ValidationTFRecordDistr3'
else:
    # Compression artifacts of recompression in distr3 seem to disturb second stage
    PATH = '/media/data/LocalizationDataNew/Output/Rev5'
LABEL_MAP_PATH = '/media/data/Dokumente/Nextcloud/Studium/Bachelorarbeit/CNNRobotLocalisation/LabelMaps/robot_label_map.pbtxt'

# Remarks on balancing:
# Identifications of robots can't be balanced due to their number. 
# This has to be considered for the evaluation total metric.
# It's not critical as the amount of images per identification only affects the resolution of the validation metric.
# Factors to balance: lighting for robot type (ground color only for natural spheros)

if STAGE == "First":
    copter_led_classes = 1
    copter_letter_classes = 1
    sphero_classes = 1
    youbot_classes = 1
else:
    copter_led_classes = 15
    copter_letter_classes = 3
    sphero_classes = 8
    youbot_classes = 3
    
summarized_ds = {
    # (balancing factor, dataset_list)
    'by_al_copter_led': (0.25*copter_led_classes,['180420_by_al_eval_copter_led.avi','180420_by_al_eval_copter_led2.avi']),
    'by_nl_copter_led': (0.25*copter_led_classes,['180420_by_nl_eval_copter_led.avi']),
    'by_al_copter_letter': (0.25*copter_letter_classes,['180420_by_al_eval_copter_A.avi','180420_by_al_eval_copter_B.avi','180420_by_al_eval_copter_C.avi']),
    'by_nl_copter_letter': (0.25*copter_letter_classes,['180420_by_nl_eval_copter_A.avi','180420_by_nl_eval_copter_B.avi','180420_by_nl_eval_copter_C.avi']),
    # not labeled..
    #'by_al_sphero': (0.5,['180413_by_al_sphero_rolling_1.avi','180413_by_al_sphero_rolling_2.avi']),
    'by_nl_sphero': (0.25*sphero_classes,['180420_by_nl_sphero_eval.avi','180420_by_nl_sphero_eval2.avi']),
    #'bg_al_sphero': (0.5*sphero_classes,['180417_bg_al_sphero_rolling.avi','180417_bg_al_sphero_rolling_distr.avi']),
    'bg_al_sphero': (0.5*sphero_classes,['180417_bg_al_sphero_rolling.avi']), #without obstacles
    #'bg_nl_sphero': (0.25*sphero_classes,['180417_bg_nl_sphero_rolling.avi','180417_bg_nl_sphero_rolling2.avi']),
    'bg_nl_sphero': (0.25*sphero_classes,['180417_bg_nl_sphero_rolling.avi']), #without obstacles
    'bg_al_youbot_mp1': (0.25*youbot_classes,['180420_by_al_youbot_A_eval.avi','180420_by_al_youbot_B_eval.avi','180420_by_al_youbot_C_eval.avi']),
    'bg_al_youbot_mp2': (0.25*youbot_classes,['180417_bg_al_youbot_mp2_A_eval.avi','180417_bg_al_youbot_mp2_B_eval.avi','180417_bg_al_youbot_mp2_C_eval.avi']),
    'bg_nl_youbot_mp2': (0.5*youbot_classes,['180417_bg_nl_youbot_mp2_A_eval.avi','180417_bg_nl_youbot_mp2_B_eval.avi','180417_bg_nl_youbot_mp2_C_eval.avi']),    
}
label_map_dict = label_map_util.get_label_map_dict(LABEL_MAP_PATH)

In [ ]:
def xml_to_example(f,dataset):
    data = parseXML(f)['annotation']
    with tf.gfile.GFile(f.replace('.xml','.jpg'), 'rb') as fid:
        encoded_image_data = fid.read()
    width = int(data['size']['width'])
    height = int(data['size']['height'])

    xmins, ymins, xmaxs, ymaxs = [],[],[],[]
    subclasses_text,subclasses,classes_text,classes = [],[],[],[]
    orientations = []
    for obj in data['object']:
        if 'undefined' in obj['name'] or ('pose_defined' in obj and obj['pose_defined']==str(0)):
            return None
        xmins.append(float(obj['bndbox']['xmin']) / width)
        ymins.append(float(obj['bndbox']['ymin']) / height)
        xmaxs.append(float(obj['bndbox']['xmax']) / width)
        ymaxs.append(float(obj['bndbox']['ymax']) / height)
        orientations.append(float(obj['pose']))
        #TODO: dist_to_cam        
        class_text = obj['name'].split('/')[0]
        classes_text.append(class_text.encode('utf8'))
        classes.append(label_map_dict[class_text])
        subclass_text = obj['name'].replace('/','_')
        subclasses_text.append(subclass_text.encode('utf8'))
        subclasses.append(label_map_dict[subclass_text])
        
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/dataset': dataset_util.bytes_feature(dataset.encode('utf8')),
        'image/filename': dataset_util.bytes_feature(data['filename'].encode('utf8')),
        'image/source_id': dataset_util.bytes_feature(data['filename'].encode('utf8')),
        'image/encoded': dataset_util.bytes_feature(encoded_image_data),
        'image/format': dataset_util.bytes_feature(b'jpg'),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/pose/orientation': dataset_util.float_list_feature(orientations),
        #'image/object/pose/iframe_w': dataset_util.float_list_feature(iframe_w),
        #'image/object/pose/iframe_h': dataset_util.float_list_feature(iframe_h),
        #'image/object/pose/dist_to_cam': dataset_util.float_list_feature(dist_to_cam),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
        'image/object/subclass/text': dataset_util.bytes_list_feature(subclasses_text),
        'image/object/subclass/label': dataset_util.int64_list_feature(subclasses),
    }))
    return tf_example

#ex = xml_to_example('/media/data/LocalizationDataNew/Output/ValidationTFRecord/180420_copter/180420_by_al_eval_copter_led.avi/BoundingBoxes/frame000700.xml')
#print(ex)

In [ ]:
if STAGE=='First':
    num_per_type = 1400
else:
    num_per_type = 110

datasets = summarized_ds
print(datasets.keys())

for i,k in enumerate(datasets.keys()):
    num_balanced = round(num_per_type * datasets[k][0])
    examples = []
    fs = get_recursive_file_list(PATH,file_matchers = datasets[k][1], file_extensions=['.xml'])
    assert len(fs) > 0
    shuffle(fs)
    for f in fs:
        if len(examples) >= num_balanced:
            break
        ex = xml_to_example(f,k)
        if ex != None:    
            examples.append(ex)
    assert (len(examples) == num_balanced), \
        'Not enough valid examples ({0} vs {1}) in dataset: {2}'.format(
            len(examples),num_balanced,k)
    shuffle(examples)
    print('Created tf record for {} with {} samples.'.format(k,num_balanced))
    writer = tf.python_io.TFRecordWriter(PATH+'/eval_{0}.record'.format(k,num_balanced))
    for example in examples:
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
# Old first stage approach

record_categories = ['copter','sphero','youbot']
num_per_cat = 200

for cat in record_categories:
    examples = []
    datasets = []
    for k in summarized_ds.keys():
        if cat in k: datasets += summarized_ds[k][1]
    print(datasets)
    # balancing of datasets for each category
    for i,d in enumerate(datasets):
        fs = get_recursive_file_list(PATH,file_matchers = [d], file_extensions=['.xml'])
        assert len(fs) > 0
        shuffle(fs)
        for f in fs:
            if len(examples) >= round((i+1)*num_per_cat/len(datasets)):
                break
            ex = xml_to_example(f)
            if ex != None:    
                examples.append(ex)
        assert (len(examples) == round((i+1)*num_per_cat/len(datasets))), \
            'Not enough valid examples ({0} vs {1}) in dataset: {2}'.format(
                len(examples),round((i+1)*num_per_cat/len(datasets)),d)
    shuffle(examples)
    writer = tf.python_io.TFRecordWriter(PATH+'/eval-firststage-'+cat+".record")
    for example in examples:
        writer.write(example.SerializeToString())
    writer.close()

In [ ]:
# Idea for second stage/final eval

num_examples_per_category = 80
summarized_ds = {
    # (categories)
    'by_al_copter_led': (15,['180420_by_al_eval_copter_led.avi','180420_by_al_eval_copter_led2.avi']),
    'by_nl_copter_led': (15,['180420_by_nl_eval_copter_led.avi']),
    #'by_al_copter_letter': (1,['180420_by_al_eval_copter_A.avi','180420_by_al_eval_copter_B.avi','180420_by_al_eval_copter_C.avi']),
    #'by_nl_copter_letter': (1,['180420_by_nl_eval_copter_A.avi','180420_by_nl_eval_copter_B.avi','180420_by_nl_eval_copter_C.avi']),
    'by_nl_sphero': (8,['180420_by_nl_sphero_eval.avi','180420_by_nl_sphero_eval2.avi']),
    'bg_al_sphero': (8,['180417_bg_al_sphero_rolling.avi','180417_bg_al_sphero_rolling_distr.avi']),
    'bg_nl_sphero': (8,['180417_bg_nl_sphero_rolling.avi','180417_bg_nl_sphero_rolling2.avi']),
    'bg_al_youbot_mp1': (1,['180420_by_al_youbot_A_eval.avi','180420_by_al_youbot_B_eval.avi','180420_by_al_youbot_C_eval.avi']),
    'bg_al_youbot_mp2': (1,['180417_bg_al_youbot_mp2_A_eval.avi','180417_bg_al_youbot_mp2_B_eval.avi','180417_bg_al_youbot_mp2_C_eval.avi']),
    'bg_nl_youbot_mp2': (1,['180417_bg_nl_youbot_mp2_A_eval.avi','180417_bg_nl_youbot_mp2_B_eval.avi','180417_bg_nl_youbot_mp2_C_eval.avi']),    
}
for k in summarized_ds.keys():
    print(k)
    num_ds = len(summarized_ds[k])
    num_ex = num_examples_per_category * summarized_ds[k][0]
    print(num_ex)
    files = get_recursive_file_list(PATH,file_matchers = summarized_ds[k][1])
    print(len(files))
    nr.choice(summarized_ds[k][1])
    
    # TODO: filter pose_defined, category_defined